In [3]:
import numpy as np
import pandas as pd
import torch
from matplotlib import pyplot as plt

print(f"cuda available: {torch.cuda.is_available()}")
print(f"torch version: {torch.__version__}")
print(f"numpy version: {np.__version__}")

cuda available: False
torch version: 2.1.0
numpy version: 1.26.0


# Hexz

A board is represented by an (N, 10, 11) numpy array. Each 10x11 slice is a one-hot encoding of the presence of specific type of piece/obstacle/etc. The slices are:

* 0: flags by P1
* 1: cell value 1-5 for P1
* 2: cells blocked for P1 (any occupied cell or a cell next to a 5)
* 3: next value for P1 
* 4: flags by P2
* 5: cell value 1-5 for P2
* 6: cells blocked for P2
* 7: next value for P2
* 8: grass cells with value 1-5

An action is specified by a (2, 10, 11) numpy array. The first 10x11 slice represents a flag move,
the second one represents a regular cell move. A flag move must have a single 1 set, a normal move
must have a single value 1-5 set.

In [4]:
def valid_idx(r_c):
    """Returns True if (r, c) = r_c represents a valid hexz board index."""
    r, c = r_c
    return r >= 0 and r < 11 and c >= 0 and c < 10-r%2


def neighbors_map():
    """Returns a dict mapping all valid (r, c) indices to their neighbor indices.
    
    The neighbor indices are represented as a 2-tuple of row and column indices."""
    result = {}
    for r in range(11):
        shift = r%2 # Depending on the row, neighbors below and above are shifted.
        for c in range(10-r%2):
            ns = filter(valid_idx, [
                (r, c+1),
                (r-1, c+shift),
                (r-1, c-1+shift),
                (r, c-1),
                (r+1, c-1+shift),
                (r+1, c+shift),
            ])
            nr, nc = zip(*ns)  # unzip
            result[(r, c)] = (np.array(nr), np.array(nc))
            
    return result

In [17]:
class Board:
    """Numpy representation of a hexz board."""

    # Used to quickly get the indices of neighbor cells.
    neighbors = neighbors_map()
    
    def __init__(self, other=None):
        if other:
            self.b = other.b.copy()
            self.nflags = list(other.nflags)
            return
        self.b = np.zeros((9, 11, 10))
        self.nflags = [3, 3]  # number of flags remaining per player
        # Even rows have 10 cells, odd rows only 9, so mark the last cell in odd rows as blocked for P1+P2.
        self.b[2, [1, 3, 5, 7, 9], 9] = 1
        self.b[6, [1, 3, 5, 7, 9], 9] = 1        
        # 2-tuple of valid indices in each slice.
        free_cells = (1 - self.b[2]).nonzero()
        # 15 randomly placed stones.
        stones = np.random.choice(np.arange(0, len(free_cells[0])), replace=False, size=15)
        self.b[2, free_cells[0][stones], free_cells[1][stones]] = 1
        self.b[6, free_cells[0][stones], free_cells[1][stones]] = 1
        free_cells = (1 - self.b[2]).nonzero()
        # 5 grass cells
        grass = np.random.choice(np.arange(0, len(free_cells[0])), replace=False, size=5)
        self.b[8, free_cells[0][grass], free_cells[1][grass]] = [1, 2, 3, 4, 5]
        self.b[2, free_cells[0][grass], free_cells[1][grass]] = 1
        self.b[6, free_cells[0][grass], free_cells[1][grass]] = 1
    
    # Helpers to retrieve slices of the board "by name".
    def flags(self, player):
        return self.b[0 + player * 4]
    def values(self, player):
        return self.b[1 + player * 4]
    def blocked(self, player):
        return self.b[2 + player * 4]
    def next_values(self, player):
        return self.b[3 + player * 4]
    def grass(self):
        return self.b[8]
    def quickview(self):
        return (self.b[0] * 8) + self.b[1] - (self.b[4] * 8) - self.b[5]
        
        
    def score(self):
        """Returns the current score as a 2-tuple."""
        return (self.b[1].sum(), self.b[5].sum())
    
    
    def result(self):
        """Returns the final result of the board.
        
        1 (player 0 won), 0 (draw), -1 (player 1 won).
        """
        p0, p1 = self.score()
        return np.sign(p0 - p1)
        
        
    def make_move(self, player, move):
        """Makes the given move.
        
        Args:
          player: 0 or 1
          move: a 4-tuple of (type, row, col, val), where type = 0 (flag) or 1 (normal)
        Does not check that it is a valid move. Should be called only
        with moves returned from `next_moves`.
        """
        typ, r, c, val = move
        self.b[typ + player*4, r, c] = val
        played_flag = typ == 0
        # Block played cell for both players.
        self.b[2, r, c] = 1
        self.b[6, r, c] = 1
        # Set next value to 0 for occupied cell.
        self.b[3, r, c] = 0
        self.b[7, r, c] = 0
        # Block neighboring cells if a 5 was played.
        nx, ny = Board.neighbors[(r, c)]
        # Update next value of neighboring cells. If we played a flag, the next value is 1.
        if played_flag:
            next_val = 1
            self.nflags[player] -= 1
        else:
            next_val = val + 1
        if next_val <= 5:
            for nr, nc in zip(nx, ny):
                if self.b[2 + player*4, nr, nc] == 0:
                    if next_val > 5:
                        self.b[3 + player*4, nr, nc] = 0
                    if self.b[3 + player*4, nr, nc] == 0:
                        self.b[3 + player*4, nr, nc] = next_val
                    elif self.b[3 + player*4, nr, nc] > next_val:
                        self.b[3 + player*4, nr, nc] = next_val
        else:
            # Played a 5: block neighboring cells and clear next value.
            self.b[2 + player*4, nx, ny] = 1
            self.b[3 + player*4, nx, ny] = 0  # Clear next value.

        # Occupy neighboring grass cells.
        if not played_flag:
            self.occupy_grass(player, r, c)
            
    
    def occupy_grass(self, player, r, c):
        """Occupies the neighboring grass cells of move_idx (a 3-tuple index into a move) for player.
        
        Expects that the move has already been played.
        """
        nx, ny = Board.neighbors[(r, c)]
        for i, j in zip(nx, ny):
            grass_val = self.b[8, i, j]
            if grass_val > 0 and grass_val <= self.b[1 + player*4, r, c]:
                # Occupy: first remove grass
                self.b[8, i, j] = 0                
                # the rest is exactly like playing a move.
                self.make_move(player, (1, r, c, grass_val))
                
        
    def next_moves(self, player):
        """Returns all possible next moves.
        
        A move is represented as a (2, 11, 10) ndarray. The first slice represents
        flag moves, the second one represents normal moves. A flag move will have exactly
        one element set to 1 in slice 0. A normal move will have exactly one element set to
        1-5 in slice 1.
        """
        moves = []
        # Do we have unoccupied cells and flags left? Then we can place another one.
        if self.nflags[player] > 0:
            # Flag in any unoccupied cell.
            rs, cs = np.nonzero(self.b[2 + player*4] == 0)  # funky way to get indices for all free cells.
            moves.extend((0, r, c, 1) for r, c in zip(rs, cs))
        # Collect all cells with a non-zero next value.
        rs, cs = np.nonzero(self.b[3 + player*4])
        moves.extend((1, r, c, self.b[3 + player*4, r, c]) for r, c in zip(rs, cs))
        return moves


In [13]:
%%timeit
b = Board()
player = 0
moves = b.next_moves(player)
num_moves = 0
while moves:
    b.make_move(player, moves[np.random.randint(len(moves))])
    num_moves += 1
    player = 1 - player
    moves = b.next_moves(player)
    if not moves:
        # No more moves for the player. See if the other player can continue.
        player = 1 - player
        moves = b.next_moves(player)
# print(f"Done after {num_moves} moves. Flags left: {b.nflags}. Score: {b.score()}")
# b.quickview()

982 µs ± 2.44 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [59]:
class Node:
    """Nodes of the MCTS tree."""
    def __init__(self, parent, player, move):
        self.parent = parent
        self.player = player
        self.move = move
        self.wins = 0.0
        self.visit_count = 0.0
        self.children = []
        
    
    def uct(self):
        if self.parent.visit_count == 0:
            return 0.5
        if self.visit_count == 0:
            win_rate = 0.5
            adjusted_count = 1
        else:
            win_rate = self.wins / self.visit_count
            adjusted_count = self.visit_count
        return (
            win_rate + 
            1.0 * np.sqrt(np.log(self.parent.visit_count) / adjusted_count)
        )
    

    def __str__(self):
        return f"Node(p={self.player}, m={self.move}, w={self.wins/self.visit_count:.3f}, n={self.visit_count}, u={self.uct():.3f}, cs={len(self.children)})"


    def __repr__(self):
        return str(self)
    
    
class MCTS:
    """Monte Carlo tree search."""
    def __init__(self, board):
        self.board = board
        self.root = Node(None, 0, None)
    
    
    def rollout(self, board, node):
        """Play a random game till the end, starting with board and player on the move."""
        player = node.player
        moves = [node.move]
        num_moves = 0
        while moves:
            board.make_move(player, moves[np.random.randint(len(moves))])
            num_moves += 1
            player = 1 - player
            moves = board.next_moves(player)
            if not moves:
                # No more moves for the player. See if the other player can continue.
                player = 1 - player
                moves = board.next_moves(player)
        return board.result()

    
    
    def backpropagate(self, node, result):
        while node:
            node.visit_count += 1
            if node.player == 0:
                node.wins += (result + 1) / 2
            else:
                node.wins += (-result + 1) / 2
            node = node.parent


    def size(self):
        q = [self.root]
        s = 0
        while q:
            n = q.pop()
            s += 1
            q.extend(n.children)
        return s
        
    
    def run(self):
        """Runs a single round of the MCTS loop."""
        b = Board(self.board)
        n = self.root
        # Find leaf node.
        while n.children:
            best = None
            best_uct = -1
            for c in n.children:
                c_uct = c.uct()
                if c_uct > best_uct:
                    best = c
                    best_uct = c_uct
            b.make_move(best.player, best.move)
            n = best
        # Reached a leaf node: expand
        player = 1 - n.player
        moves = b.next_moves(player)
        if not moves:
            # No more moves for player. Try other player.
            player = 1 - player
            moves = b.next_moves(player)
        if not moves:
            # Game is over
            self.backpropagate(n, b.result())
            return
        # Rollout time!
        for move in moves:
            n.children.append(Node(n, player, move))
        c = n.children[np.random.randint(len(n.children))]
        result = self.rollout(b, c)
        self.backpropagate(c, result)
        

In [65]:
# %%timeit
b = Board()
m = MCTS(b)
for _ in range(1000):
    m.run()
m.root.children

[Node(p=1, m=(0, 0, 0, 1), w=0.500, n=10.0, u=1.331, cs=84),
 Node(p=1, m=(0, 0, 1, 1), w=0.167, n=6.0, u=1.240, cs=84),
 Node(p=1, m=(0, 0, 2, 1), w=0.647, n=17.0, u=1.285, cs=84),
 Node(p=1, m=(0, 0, 3, 1), w=0.647, n=17.0, u=1.285, cs=84),
 Node(p=1, m=(0, 0, 5, 1), w=0.500, n=10.0, u=1.331, cs=84),
 Node(p=1, m=(0, 0, 6, 1), w=0.000, n=4.0, u=1.314, cs=84),
 Node(p=1, m=(0, 0, 7, 1), w=0.375, n=8.0, u=1.304, cs=84),
 Node(p=1, m=(0, 0, 8, 1), w=0.444, n=9.0, u=1.321, cs=84),
 Node(p=1, m=(0, 0, 9, 1), w=0.375, n=8.0, u=1.304, cs=84),
 Node(p=1, m=(0, 1, 0, 1), w=0.000, n=4.0, u=1.314, cs=84),
 Node(p=1, m=(0, 1, 1, 1), w=0.375, n=8.0, u=1.304, cs=84),
 Node(p=1, m=(0, 1, 3, 1), w=0.667, n=18.0, u=1.286, cs=84),
 Node(p=1, m=(0, 1, 4, 1), w=0.647, n=17.0, u=1.285, cs=84),
 Node(p=1, m=(0, 1, 8, 1), w=0.500, n=10.0, u=1.331, cs=84),
 Node(p=1, m=(0, 2, 1, 1), w=0.000, n=4.0, u=1.314, cs=84),
 Node(p=1, m=(0, 2, 2, 1), w=0.375, n=8.0, u=1.304, cs=84),
 Node(p=1, m=(0, 2, 3, 1), w=0.44